In [ ]:
# normal pre processing + count vectorizer + SVC

In [6]:
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

df = pd.read_csv("train.csv", lineterminator='\n')

In [7]:
print(df.shape)

df.dropna(subset = ["author"], inplace=True) # removing the nan rows for author
print(df.shape)

df.dropna(subset = ["title"], inplace=True) # removing the null rows for title


print(df.shape)

(20800, 5)
(18843, 5)
(18285, 5)


In [8]:
import spacy
# This code is almost exactly the same as the one in my lab 1
# removing stop words
nlp = spacy.load('en_core_web_sm')


def tokens(text):
    doc = nlp(text)
    return [t.lemma_ for t in doc if not t.is_stop and t.is_alpha]

def preprocess(text):
    # TODO: Replace the next line with your own code.
    return tokens(text)

# lemmatization
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

In [9]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer # for vectorizing multiple columns at the same time

vectorizer = CountVectorizer(tokenizer=preprocess)

X_train = df_train[["title", "author", "text"]]
X_test = df_test[["title", "author", "text"]]


ct = make_column_transformer((vectorizer, "title"), (vectorizer, "author"), (vectorizer, "text") )

X_train_vectorized = ct.fit_transform(X_train)
X_test_vectorized = ct.transform(X_test)


y_train = df_train.iloc[:, 4:5]
y_test = df_test.iloc[:, 4:5]

In [15]:
from scipy import sparse

sparse.save_npz("X_train_count_vectorized.npz", X_train_vectorized)
sparse.save_npz("X_test_count_vectorized.npz", X_test_vectorized)


In [16]:
import pandas

y_train.to_pickle("y_count_train")
y_test.to_pickle("y_count_test")



In [17]:
from sklearn.svm import SVC
clf = SVC()

In [18]:

X_train_vectorized_df = pd.DataFrame(X_train_vectorized.toarray())
X_test_vectorized_df = pd.DataFrame(X_test_vectorized.toarray())

In [19]:
clf.fit(X_train_vectorized, y_train)

/home/louda622/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [20]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test_vectorized_df)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2094
           1       0.94      0.96      0.95      1563

    accuracy                           0.96      3657
   macro avg       0.96      0.96      0.96      3657
weighted avg       0.96      0.96      0.96      3657

